Environment Configurations

In [ ]:
# mount to drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# installations
import sys
!{sys.executable} -m pip install segmentation-models-pytorch
import sys
!{sys.executable} -m pip install -U albumentations --no-binary qudida,albumentations
import albumentations as albu
print(albu.__version__)

!pip3 install tensorflow==2.2
!pip3 install keras==2.3.1
!pip3 install -U segmentation-models

!{sys.executable} -m pip install opencv-python-headless==4.1.2.30
#!{sys.executable} pip install -U git+https://github.com/albu/albumentations --no-cache-dir
#!{sys.executable} -m pip install --force-reinstall albumentations==1.0.3
#!{sys.executable} -m pip install opencv-python
#!{sys.executable} -m pip uninstall folium
#!{sys.executable} -m pip install folium==0.2.1
#!{sys.executable} -m pip uninstall numpy
#!{sys.executable} -m pip install numpy==1.18.5

In [ ]:
# Imports
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import tensorflow as tf
import tensorflow.keras as keras
import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from tensorflow.keras.metrics import MeanIoU
from keras.models import load_model

# Config gpu growth
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

Getting stored data

In [ ]:
# Data directories
data_dir = "drive/MyDrive/PlusDS/"
x_train_dir = os.path.join(data_dir,'ISIC2018_Task1-2_Training_Input')
y_train_dir = os.path.join(data_dir,'ISIC2018_Task2_Training_GroundTruth_v3')

x_val_dir = os.path.join(data_dir,'ISIC2018_Task1-2_Validation_Input')
y_val_dir = os.path.join(data_dir,'ISIC2018_Task2_Validation_GroundTruth')

x_test_dir = os.path.join(data_dir,'ISIC2018_Task1-2_Test_Input')

# Used Functions

In [ ]:
from numpy.lib.twodim_base import mask_indices
# Data processing
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """

    WIDTH = 256
    HEIGHT = 256
    CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
    def __init__(
            self, 
            images_dir,
            masks_dir=None, 
            attribute=None, 
            augmentation=None, 
            preprocessing=None,
            num_images = None,
    ):
        
        
        self.ids = [id for id in os.listdir(images_dir) if id[-4:] == ".jpg"]
        if num_images == None:
          num_images = len(self.ids)
        self.ids = sorted(self.ids)[:num_images]
        self.images = [os.path.join(images_dir, image_id) for image_id in self.ids]
        print(len(self.ids))
        if masks_dir != None:
          self.masks = {}
          if attribute!=None:
            self.masks[attribute] = [os.path.join(masks_dir, image_id[:-4]+"_attribute_"+attribute+".png") for image_id in self.ids]
          else:
            for attribute in CLASSES:
              self.masks[attribute] = [os.path.join(masks_dir, image_id[:-4]+"_attribute_"+attribute+".png") for image_id in self.ids]
        else:
          self.masks = None
        #self.globules_masks = [os.path.join(masks_dir, image_id[:-4]+"_attribute_globules.png") for image_id in self.ids]
        #self.pigment_network_masks = [os.path.join(masks_dir, image_id[:-4]+"_attribute_pigment_network.png") for image_id in self.ids]
        #self.milia_like_cyst_masks = [os.path.join(masks_dir, image_id[:-4]+"_attribute_milia_like_cyst.png") for image_id in self.ids]
        #self.negative_network_masks = [os.path.join(masks_dir, image_id[:-4]+"_attribute_negative_network.png") for image_id in self.ids]
        #self.streaks_masks = [os.path.join(masks_dir, image_id[:-4]+"_attribute_streaks.png") for image_id in self.ids]

        # convert str names to class values on masks
        #self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.WIDTH, self.HEIGHT), interpolation = cv2.INTER_AREA)
        if self.masks == None:
          return image
        else:
          mask = {}
          for attribute in self.masks.keys():
            mask[attribute] = cv2.imread(self.masks[attribute][i], 0)
            mask[attribute] = cv2.resize(mask[attribute], (self.WIDTH, self.HEIGHT), interpolation = cv2.INTER_AREA)

        #globules_mask = cv2.imread(self.globules_masks[i], 0)
        #pigment_network_mask = cv2.imread(self.pigment_network_masks[i], 0)
        #milia_like_cyst_mask = cv2.imread(self.milia_like_cyst_masks[i], 0)
        #negative_network_mask = cv2.imread(self.negative_network_masks[i], 0)
        #streaks_mask = cv2.imread(self.streaks_masks[i], 0)

        # masks = dict of five masks
        #masks = {"globules_mask": globules_mask,
         #        "pigment_network_mask": pigment_network_mask,
          #       "milia_like_cyst_mask": milia_like_cyst_mask,
           #      "negative_network_mask": negative_network_mask,
            #     "streaks_mask": streaks_mask}

        
        # extract certain classes from mask (e.g. cars)
        #for mask_key in masks.keys():
        #    mask_list = [masks[mask_key]]
        #    masks[mask_key] = np.stack(mask_list, axis=-1).astype('float')
        #mask = np.stack(masks, axis=-1).astype('float')

        # extract certain classes from mask (e.g. cars)
        #masks = [mask]
        #mask = np.stack(masks, axis=-1).astype('float')

        # apply augmentations
        #if self.augmentation:
            #sample = self.augmentation(image=image, mask=mask)
            #image, mask = sample['image'], sample['mask']
            #sample = self.augmentation(image=image,mask=masks["globules_mask"])
            #image = sample['image']
            #for mask_key in masks.keys():
            #    sample = self.augmentation(image=image,mask=masks[mask_key])
            #    masks[mask_key] = sample['mask']
        
        # apply preprocessing
        #if self.preprocessing:
            #sample = self.preprocessing(image=image, mask=mask)
            #image, mask = sample['image'], sample['mask']
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [ ]:
# Getting data
def get_inputs(attribute=None, images_dir=None, masks_dir=None,num_images=None):
  CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
  # Training Data
  dataset = Dataset(images_dir=images_dir, masks_dir=masks_dir,num_images=num_images,attribute=attribute)
  if masks_dir == None:
    x_train = []
    for i in range(len(dataset)):
      image = dataset[i]
      x_train.append(image)
    x_train = np.concatenate( [x_train], axis=0 )
    # Adding dimension
    #  ONLY IF GRAYSCALED
    #x_train = np.expand_dims(x_train, axis=-1)
    # Normalization
    x_train = x_train /255
    return x_train
  if attribute != None:
    x_train = []
    y_train = []

    for i in range(len(dataset)):
      image, mask = dataset[i]
      x_train.append(image)
      y_train.append(mask[attribute])
    x_train = np.concatenate( [x_train], axis=0 )
    y_train = np.concatenate( [y_train], axis=0 )
    # Adding dimension
    y_train = np.expand_dims(y_train, axis=-1)
    #  ONLY IF GRAYSCALED
    #x_train = np.expand_dims(x_train, axis=-1)
    # Normalization
    x_train = x_train /255
    y_train = y_train/255
    return x_train, y_train
  else:
    x_train = []
    y_train = {}
    for attribute in CLASSES:
        y_train[attribute] = []
    #y_train_globules = []
    #y_train_milia = []
    #y_train_pigment = []
    #y_train_negative = []
    #y_train_streaks = []
    for i in range(len(dataset)):
      image, mask = dataset[i]
      for attribute in CLASSES:
        y_train[attribute].append(mask[attribute])
      x_train.append(image)
      #y_train_globules.append(mask["globules"])
      #y_train_milia.append(mask["milia_like_cyst"])
      #y_train_pigment.append(mask["pigment_network"])
      #y_train_negative.append(mask["negative_network"])
      #y_train_streaks.append(mask["streaks"])
    x_train = np.concatenate( [x_train], axis=0 )
    for attribute in CLASSES:
        y_train[attribute] = np.concatenate( [y_train[attribute]], axis=0 )
        y_train[attribute] = np.expand_dims(y_train[attribute], axis=-1)
        y_train[attribute] = y_train[attribute]/255
    #y_train_globules = np.concatenate( [y_train_globules], axis=0 )
    #y_train_milia = np.concatenate( [y_train_milia], axis=0 )
    #y_train_pigment = np.concatenate( [y_train_pigment], axis=0 )
    #y_train_negative = np.concatenate( [y_train_negative], axis=0 )
    #y_train_streaks = np.concatenate( [y_train_streaks], axis=0 )
    # Adding dimension
    #y_train_globules = np.expand_dims(y_train_globules, axis=-1)
    #y_train_milia = np.expand_dims(y_train_milia, axis=-1)
    #y_train_pigment = np.expand_dims(y_train_pigment, axis=-1)
    #y_train_negative = np.expand_dims(y_train_negative, axis=-1)
    #y_train_streaks = np.expand_dims(y_train_streaks, axis=-1)
    #  ONLY IF GRAYSCALED
    #x_train = np.expand_dims(x_train, axis=-1)
    # Normalization
    x_train = x_train /255
    #y_train_globules = y_train_globules/255
    #y_train_milia = y_train_milia/255
    #y_train_pigment = y_train_pigment/255
    #y_train_negative = y_train_negative/255
    #y_train_streaks = y_train_streaks/255
    return x_train, y_train #y_train_globules, y_train_milia, y_train_pigment, y_train_negative, y_train_streaks

In [ ]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    # applies sliding filters each with 3 kernels (rgb) to the input, to get average value of the window as a single value in output
    # padding = "same" so 0's are padded around the input to make output the same size as input  
    x = Conv2D(num_filters, 3, padding="same")(input)

    # standardizes output to have mean of 0 and std of 1
    x = BatchNormalization()(x)   #Not in the original network. 
    
    # outputs negative input values as 0, outputs positive inputs directly
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    # downsizes output to only have max value in every 2x2 square
    p = MaxPool2D((2, 2))(x)

    # x is convolution output, p is maxpool of the convolution
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    # reverse convolution
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    # concatenate to the skipped convolution output of the same dimensions (1 layer before input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes,filter_rate,dropout):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, int(64*filter_rate))
    s2, p2 = encoder_block(p1, int(128*filter_rate))
    s3, p3 = encoder_block(p2, int(256*filter_rate))
    s4, p4 = encoder_block(p3, int(512*filter_rate))

    b1 = conv_block(p4, int(1024*filter_rate)) #Bridge

    # add dropout, 0.2
    # randomly drops inputs with frequency of the rate, scales remaining input up by 1/(1-rate)
    b1 = Dropout(dropout)(b1)
    
    d1 = decoder_block(b1, s4, int(512*filter_rate))
    d2 = decoder_block(d1, s3, int(256*filter_rate))
    d3 = decoder_block(d2, s2, int(128*filter_rate))
    d4 = decoder_block(d3, s1, int(64*filter_rate))

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

#Build Unet using aditional encoder and decoder block
def build_unet_new(input_shape, n_classes,filter_rate):
    inputs = Input(input_shape)

    # NEW
    s1, p1 = encoder_block(inputs, int(64*filter_rate))
    s2, p2 = encoder_block(p1, int(128*filter_rate))
    s3, p3 = encoder_block(p2, int(256*filter_rate))
    s4, p4 = encoder_block(p3, int(512*filter_rate))
    s5, p5 = encoder_block(p4, int(512*filter_rate))

    b1 = conv_block(p5, int(1024*filter_rate)) #Bridge

    # add dropout, 0.2
    b1 = Dropout(.2)(b1)

    d1 = decoder_block(b1, s5, int(512*filter_rate))
    d2 = decoder_block(d1, s4, int(512*filter_rate))
    d3 = decoder_block(d2, s3, int(256*filter_rate))
    d4 = decoder_block(d3, s2, int(128*filter_rate))
    d5 = decoder_block(d4, s1, int(64*filter_rate))

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d5)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)+1
    plt.figure(figsize=(16, 5))
    overlay_imgs = []
    for i, (name, image) in enumerate(images.items()):
        if i == 0 or i == 1:
          overlay_imgs.append(image)
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.subplot(1,n,n)
    plt.xticks([])
    plt.yticks([])
    plt.title("Overlay")
    plt.imshow(overlay_imgs[0])
    plt.imshow(overlay_imgs[1], alpha=0.5)
    #plt.save_fig("/content/drive/My Drive/PlusDS/visualizations/")
    plt.show()

In [ ]:
def overlay(image, mask):
  plt.figure(figsize=(5,5))
  plt.imshow(image)
  plt.imshow(mask, alpha=0.5)
  plt.show()

In [ ]:
# Thresholding helper
def getThresholded(y, threshold):
  y = y > threshold
  y = y*1
  return y

In [ ]:
# Jaccard Index/ IOU
# DEPRACATED
def getMeanIOU(y_val_thresholded, y_val_pred_thresholded):
  n_classes = 2
  IOU_keras = MeanIoU(num_classes=n_classes)  
  IOU_keras.update_state(y_val_thresholded, y_val_pred_thresholded)
  meanIOU = IOU_keras.result().numpy()
  return meanIOU

In [ ]:
# Dice Coefficient
def getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded):
  dice_coeffs = []
  for i in range(len(y_val_pred_thresholded)):
    im_sum = y_val_pred_thresholded[i].sum() + y_val_thresholded[i].sum()
    if im_sum != 0:
      # Compute Dice coefficient
      intersection = np.logical_and(y_val_pred_thresholded[i], y_val_thresholded[i])
      dice_coeffs.append(2.0 * intersection.sum() / im_sum)
    else:
      dice_coeffs.append(0)
  return dice_coeffs

In [ ]:
# IOUs
def getIOUs(y_val_thresholded, y_val_pred_thresholded):
  ious = []
  for i in range(len(y_val_pred_thresholded)):
    union = np.logical_or(y_val_thresholded[i], y_val_pred_thresholded[i]).sum()
    if union != 0:
      # Compute IOU
      intersection = np.logical_and(y_val_pred_thresholded[i], y_val_thresholded[i])
      ious.append(intersection.sum() / union)
    else:
      ious.append(0)
  return ious

In [ ]:
# Plotting function for test prediction output
def plot(test_img, thresholded_mask, output_path):
  test_img = test_img.squeeze()
  thresholded_mask = thresholded_mask.squeeze()
  plt.figure(figsize=(16, 8))
  plt.subplot(221)
  plt.title('Testing Image')
  plt.imshow(test_img)
  plt.subplot(222)
  plt.title('Prediction on test image')
  plt.imshow(thresholded_mask, cmap='gray')

  plt.savefig(output_path)
  #plt.show()

In [ ]:
def filter(images, masks):
  filtered_images = []
  filtered_masks = []
  for i in range(len(masks)):
    mask = masks[i]
    image = images[i]
    sum = np.sum(mask)
    if sum > 0:
      filtered_images.append(image)
      filtered_masks.append(mask)
  filtered_images = np.concatenate( [filtered_images], axis=0 )
  filtered_masks = np.concatenate( [filtered_masks], axis=0 )
  return filtered_images, filtered_masks

In [ ]:
def calculateThreshold(masks):
  sum = 0
  for mask in masks:
    sum += np.sum(mask)
  return sum / (len(masks) * len(masks[0]) * len(masks[0][0]))

In [ ]:
def setUpSummaryFile():
  f = open("/content/drive/My Drive/PlusDS/summaryNew.txt","w")
  f.write("Mean IOU:"+"\n")
  attributes = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
  for attribute in attributes:
    f.write(attribute+"\n")
  f.write("\n"+"Mean Dice Coeff: "+"\n")
  for attribute in attributes:
    f.write(attribute+"\n")
  f.close()

In [ ]:
def writeSummary(new_line, attribute, score):
  a_file = open("/content/drive/My Drive/PlusDS/summaryNew.txt", "r")

  lines = a_file.readlines()
  a_file.close()

  new_file = open("/content/drive/My Drive/PlusDS/summaryNew.txt", "w")
  sectionfound = False
  for line in lines:
    new_file.write(line)
    if line.strip(": \n") == score:
      sectionfound = True
      print("in")
    if line.strip("\n") == attribute and sectionfound:
      new_file.write(new_line+"\n")
      sectionfound = False
  new_file.close()

In [ ]:
def delete_line(lines_to_delete):
  a_file = open("/content/drive/My Drive/PlusDS/summaryNew.txt", "r")
  lines = a_file.readlines()
  a_file.close()
  new_file = open("/content/drive/My Drive/PlusDS/summaryNew.txt", "w")
  for line in lines:
    if line not in lines_to_delete:
      new_file.write(line)
  new_file.close()

# Initial Data Processing and Saving DON"T HAVE TO RUN AGAIN :)

In [ ]:
CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
INPUT_SHAPE = (256, 256, 3)
NUM_TRAIN_IMAGES = 2594
NUM_VAL_IMAGES = 500
NUM_TEST_IMAGES = 100

# Processing training images and masks
x_train, y_train_dict = get_inputs(images_dir=x_val_dir,masks_dir=y_val_dir,num_images=NUM_VAL_IMAGES)

np.save('/content/drive/My Drive/PlusDS/processedData/data.npy', x_train) # save

for attribute in y_train_dict.keys():
  np.save('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy',y_train_dict[attribute])

# Processing validation images and masks
x_val, y_val_dict = get_inputs(images_dir=x_val_dir,masks_dir=y_val_dir,num_images=NUM_VAL_IMAGES)
np.save('/content/drive/My Drive/PlusDS/processedData/val_data.npy', x_val) # save
for attribute in y_val_dict.keys():
  np.save('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy',y_val_dict[attribute])

# Processing test images
x_test = get_inputs(images_dir=x_test_dir,num_images=NUM_TEST_IMAGES)
np.save('/content/drive/My Drive/PlusDS/processedData/test_data.npy', x_test) # save

In [ ]:
# Confirm processed data was saved and loaded correctly
x_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/data.npy') # load
print(np.array_equal(x_train,x_train_loaded))
x_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/val_data.npy')
print(np.array_equal(x_val,x_val_loaded))
x_test_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/test_data.npy')
print(np.array_equal(x_test,x_test_loaded))
for attribute in CLASSES:
  y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy') # load
  print(np.array_equal(y_train_dict[attribute],y_train_loaded))
  y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')
  print(np.array_equal(y_val_dict[attribute],y_val_loaded))

# **Main Looper for training and predicting all five attributes**

In [ ]:
CLASSES = ['globules','milia_like_cyst',
           'pigment_network',
           'negative_network','streaks'
           ]
INPUT_SHAPE = (256, 256, 3)
NUM_TRAIN_IMAGES = 2594
NUM_VAL_IMAGES = 100
NUM_TEST_IMAGES = 100

# load processed data
x_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/data.npy') 
x_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/val_data.npy')
x_test_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/test_data.npy')


# Trying different learning rate and filter rate configurations
LR_FOLDERS = ["lr_-2", "lr_-3", 
              "lr_-4"
              "lr_-5"
              ]
LRS = [1e-2, 1e-3, 
       1e-4,
       1e-5
       ]
FILTER_FOLDERS = ["full_filters_0.1_dropout", 
                  "half_filters", "quarter_filters"
                  ]
FILTER_RATES = [2,
                1, 
                0.5, 0.25
                ]
DROPOUT = 0.2

In [ ]:
# LOOPER For different LR and FILTER_RATE combinations for each attribute
for i in range(len(LRS)):
  for j in range(len(FILTER_RATES)):

    LEARNING_RATE_FOLDER = LR_FOLDERS[i]
    LEARNING_RATE = LRS[i]
    FILTER_FOLDER = FILTER_FOLDERS[j]
    FILTER_RATE = FILTER_RATES[j]

    for attribute in CLASSES:
      print("Currently runnning for "+attribute)
      y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy') # load processed data
      y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')

      x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded, y_train_loaded)
      x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded, y_val_loaded)

      # Define model
      model = build_unet(INPUT_SHAPE, n_classes=1,filter_rate=FILTER_RATE,dropout=DROPOUT)

      # NEW COMPILE
      # bce_jaccard_loss only considers the positive pixels (intersection over union)
      model.compile(optimizer=Adam(learning_rate = LEARNING_RATE), loss=bce_jaccard_loss, metrics=[iou_score])

      # OLD COMPILE
      # binary_crossentropy looks at all pixel values compared to ground truth
      #model.compile(optimizer=Adam(learning_rate = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])

      # fit model
      history = model.fit(
        x=x_train_loaded_filtered,
        y=y_train_loaded_filtered,
        batch_size=16, 
        epochs=5,
        validation_data=(x_val_loaded_filtered, y_val_loaded_filtered),
      )

      # save model
      model.save("/content/drive/My Drive/PlusDS/savedModels/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"Model.h5")

      #IOU
      y_val_pred = model.predict(x_val_loaded_filtered)
      threshold = calculateThreshold(y_train_loaded_filtered)
      y_val_pred_thresholded = getThresholded(y_val_pred, threshold)
      y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

      # IOU
      ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
      meanIOU = np.mean(ious)
      print("Mean IoU =", meanIOU)
      # Boxplot
      plt.boxplot(ious)
      plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"IOU.png")
      plt.show()


      # Write meanIOU to summary text
      writeSummary(LEARNING_RATE_FOLDER+" "+FILTER_FOLDER+" "+str(meanIOU), attribute, "Mean IOU")
      #writeSummary("Original: "+str(meanIOU), "Mean IOU")

      # Dice Scores
      dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
      # DICE
      dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
      meanDice = np.mean(dice_coeffs)
      print("Mean Dice Coeff: ", meanDice)
      # Boxplot
      plt.boxplot(dice_coeffs)
      plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"Dice.png")
      plt.show()
      # Write dice score mean to new summary text
      writeSummary(LEARNING_RATE_FOLDER+" "+FILTER_FOLDER+" "+str(meanDice), attribute, "Mean Dice Coeff")

      for y in range(len(y_val_pred)):
        visualize(image=x_val_loaded_filtered[y].squeeze(),mask=y_val_loaded_filtered[y].squeeze(),prediction=y_val_pred[y].squeeze(),thresholded_mask=y_val_pred_thresholded[y].squeeze())
        print(sorted([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"])[y])

      # Test set predictions
      """
      y_test_pred = model.predict(x_test_loaded)
      for i in range(len(y_test_pred)):
        test_img = x_test_loaded[i]
        predicted_mask = y_test_pred[i]
        thresholded_mask = getThresholded(predicted_mask, threshold)
        # visualize test predictions
        plot(test_img, thresholded_mask, "/content/drive/My Drive/PlusDS/plots/predictedMasks/"+str(i)+"_"+attribute+".png")
      """


## Pigment Network Model With Additional Encoder and Decoder Blocks

In [ ]:
# FOR ADDITIONAL ENCODER AND DECODER BLOCKS UNET FOR PIGMENT_NETWORK
#CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
INPUT_SHAPE = (256, 256, 3)
NUM_TRAIN_IMAGES = 2594
NUM_VAL_IMAGES = 100
NUM_TEST_IMAGES = 100

x_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/data.npy') # load processed data
x_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/val_data.npy')
x_test_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/test_data.npy')

LEARNING_RATE = 1e-4
FILTER_RATE = 1
attribute = "pigment_network"

print("Currently runnning for "+attribute)
y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy') # load processed data
y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')

x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded, y_train_loaded)
x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded, y_val_loaded)

In [ ]:
#TRAINING MODEL

# Define new model (additional encoder and decoder block)
model = build_unet_new(INPUT_SHAPE, n_classes=1,filter_rate=FILTER_RATE)
model.compile(optimizer=Adam(learning_rate = LEARNING_RATE), loss=bce_jaccard_loss, metrics=[iou_score])

# fit model
history = model.fit(
  x=x_train_loaded_filtered,
  y=y_train_loaded_filtered,
  batch_size=16,
  epochs=5,
  validation_data=(x_val_loaded_filtered, y_val_loaded_filtered),
)

# save model
model.save("/content/drive/My Drive/PlusDS/savedModels/lr_-4/full_filters_dropout"+"/"+attribute+"Model.h5")


In [ ]:
test = "lr_-4/full_filters_dropout"

# Load Model
model = load_model("/content/drive/My Drive/PlusDS/savedModels/"+test+"/"+attribute+"Model.h5", compile=False)

# Predict
y_val_pred = model.predict(x_val_loaded_filtered)
threshold = calculateThreshold(y_train_loaded_filtered)
y_val_pred_thresholded = getThresholded(y_val_pred, threshold)
y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

# IOU
ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
meanIOU = np.mean(ious)
print("Mean IoU =", meanIOU)
# Boxplot
plt.boxplot(ious)
plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+test+"/"+attribute+"IOU.png")
plt.show()
# Write meanIOU to new summary text
writeSummary("lr_-4 full_filters Dropout: "+str(meanIOU), attribute, "Mean IOU")

# DICE
dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
meanDice = np.mean(dice_coeffs)
print("Mean Dice Coeff: ", meanDice)
# Boxplot
plt.boxplot(dice_coeffs)
plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+test+"/"+attribute+"Dice.png")
plt.show()
# Write dice score mean to new summary text
writeSummary("lr_-4 full_filters Dropout: "+str(meanDice), attribute, "Mean Dice Coeff")

# Visualize predictions
for y in range(len(y_val_pred)):
  visualize(image=x_val_loaded_filtered[y].squeeze(),mask=y_val_loaded_filtered[y].squeeze(),prediction=y_val_pred[y].squeeze(),thresholded_mask=y_val_pred_thresholded[y].squeeze())
  print(sorted([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"])[y])

In [ ]:
#np.save('/content/drive/My Drive/PlusDS/processedData/val_predictions.npy', y_val_pred) # save
#np.save('/content/drive/My Drive/PlusDS/processedData/test_predictions.npy', y_test_pred) # save


# Running Predictions and Visualizations with Saved Models

In [ ]:
# just looking at IOU boxplot and mean for now
def load_and_display(test, thresholdFactor):
  print(test)
  # Load Model
  model = load_model("/content/drive/My Drive/PlusDS/savedModels/"+test+"/"+attribute+"Model.h5", compile=False)

  # Predict
  y_val_pred = model.predict(x_val_loaded_filtered)
  threshold = calculateThreshold(y_train_loaded_filtered)
  threshold = threshold * thresholdFactor
  y_val_pred_thresholded = getThresholded(y_val_pred, threshold)
  y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

  # IOU
  ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
  meanIOU = np.mean(ious)
  print("Mean IoU =", meanIOU)
  # Boxplot
  plt.boxplot(ious)
  #plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+test+"/"+attribute+"IOU.png")
  plt.show()
  # Write meanIOU to new summary text
  #writeSummary("Additional Block Dropout: "+str(meanIOU), attribute, "Mean IOU")

  # DICE
  dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
  meanDice = np.mean(dice_coeffs)
  print("Mean Dice Coeff: ", meanDice)
  # Boxplot
  #plt.boxplot(dice_coeffs)
  #plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+test+"/"+attribute+"Dice.png")
  #plt.show()
  # Write dice score mean to new summary text
  #writeSummary("Additional Block Dropout: "+str(meanDice), attribute, "Mean Dice Coeff")

  # Visualize predictions
  #for y in range(len(y_val_pred)):
  #  visualize(image=x_val_loaded_filtered[y].squeeze(),mask=y_val_loaded_filtered[y].squeeze(),prediction=y_val_pred[y].squeeze(),thresholded_mask=y_val_pred_thresholded[y].squeeze())
  #  print(sorted([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"])[y])

In [ ]:
# Loading x data
x_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/val_data.npy')
x_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/data.npy')

attribute = "pigment_network"
y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')
y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy')

x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded,y_train_loaded)
x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded,y_val_loaded)

In [ ]:
# Tried:
# Play with threshold- best one still full_filters_dropout
# lr_-5: worse
# 1.2_filters still worse even with optimal threshold 

# Next:
# maybe try putting dropout before batchnorm in conv block? or more dropout?

load_and_display("lr_-4/full_filters_0.1_dropout",1)
load_and_display("lr_-4/full_filters_0.1_dropout",1.5)
load_and_display("lr_-4/full_filters_0.1_dropout",1.6)

In [ ]:
CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
for attribute in CLASSES:
  y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')
  y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy')

  x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded,y_train_loaded)
  x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded,y_val_loaded)

  threshold = calculateThreshold(y_train_loaded_filtered)
  print("Threshold: "+str(threshold))

  model = load_model("drive/MyDrive/PlusDS/savedModels/original/"+attribute+"Model.h5", compile=False)
      
  y_val_pred = model.predict(x_val_loaded_filtered)

  y_val_pred_thresholded = getThresholded(y_val_pred, threshold)

  y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

  ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
  meanIOU = np.mean(ious)
  print("Mean IOU: ", meanIOU)
  plt.boxplot(ious)
  plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/original/"+attribute+"IOU.png")
  plt.show()

  dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
  meanDice = np.mean(dice_coeffs)
  print("Mean Dice Coeff: ", meanDice)
  plt.boxplot(meanDice)
  plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/original/"+attribute+"Dice.png")
  plt.show()

  writeSummary("Original: "+str(meanIOU), attribute, "Mean IOU:")
  writeSummary("Original: "+str(meanDice), attribute, "Mean Dice Coeff:")

In [ ]:
CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
#Load previously saved models
"""
attribute_models = {'globules': ['lr_-3','full_filters'],
                    'milia_like_cyst': ['lr_-2','quarter_filters'],
                    'pigment_network': ['lr_-4','full_filters'],
                    'negative_network': ['lr_-2','half_filters'],
                    'streaks': ['lr_-2','quarter_filters']}
"""
original_models = {}
#optimal_models = {}
LR_FOLDERS = ["lr_-2", "lr_-3", "lr_-4"]
#LRS = [1e-2, 1e-3, 1e-4]
FILTER_FOLDERS = ["full_filters", "half_filters", "quarter_filters"]
#FILTER_RATES = [1, 0.5, 0.25]

for attribute in CLASSES:
      
  y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')
  y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy')

  x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded,y_train_loaded)
  x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded,y_val_loaded)

  threshold = calculateThreshold(y_train_loaded_filtered)
  print("Threshold: "+str(threshold))

  for i in range(len(LR_FOLDERS)):
    for j in range(len(FILTER_FOLDERS)):
      LEARNING_RATE_FOLDER = LR_FOLDERS[i]
      FILTER_FOLDER = FILTER_FOLDERS[j]

      model = load_model("drive/MyDrive/PlusDS/savedModels/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"Model.h5", compile=False)
      
      y_val_pred = model.predict(x_val_loaded_filtered)

      y_val_pred_thresholded = getThresholded(y_val_pred, threshold)

      y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

      ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
      meanIOU = np.mean(ious)
      print("Mean IOU: ", meanIOU)
      plt.boxplot(ious)
      plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"IOU.png")
      plt.show()

      dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
      meanDice = np.mean(dice_coeffs)
      print("Mean Dice Coeff: ", meanDice)
      plt.boxplot(meanDice)
      plt.savefig("/content/drive/My Drive/PlusDS/plots/diceCoeffs/"+LEARNING_RATE_FOLDER+"/"+FILTER_FOLDER+"/"+attribute+"Dice.png")
      plt.show()

      writeSummary(LEARNING_RATE_FOLDER+" "+FILTER_FOLDER+" "+str(meanIOU), attribute, "Mean IOU:")
      writeSummary(LEARNING_RATE_FOLDER+" "+FILTER_FOLDER+" "+str(meanDice), attribute, "Mean Dice Coeff:")

for attribute in CLASSES:
  original_models[attribute] = load_model("drive/MyDrive/PlusDS/savedModels/original/"+attribute+"Model.h5", compile=False)
  #optimal_models[attribute] = load_model('drive/MyDrive/PlusDS/savedModels/'+attribute_models[attribute][0]+'/'+attribute_models[attribute][1]+'/'+attribute+"Model.h5", compile=False)
#globules_model = load_model("drive/MyDrive/PlusDS/savedModels/lr_-2/half_filters/globules.h5", compile=False)

In [ ]:

CLASSES = ['globules','milia_like_cyst','pigment_network','negative_network','streaks']
attribute_models = {'globules': ['lr_-2','quarter_filters'],
                    'milia_like_cyst': ['lr_-2','quarter_filters'],
                    'pigment_network': ['lr_-4','full_filters'],
                    'negative_network': ['lr_-2','full_filters'],
                    'streaks': ['lr_-3','half_filters']}

original_models = {}
optimal_models = {}
for attribute in CLASSES:
  original_models[attribute] = load_model("drive/MyDrive/PlusDS/savedModels/original/"+attribute+"Model.h5", compile=False)
  optimal_models[attribute] = load_model('drive/MyDrive/PlusDS/savedModels/'+attribute_models[attribute][0]+'/'+attribute_models[attribute][1]+'/'+attribute+"Model.h5", compile=False)

In [ ]:
def visualize_attribute(attribute):
  print("Running for "+attribute+" with optimal configurations "+attribute_models[attribute][0]+" and "+attribute_models[attribute][1]+"\n")
  y_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_val_mask.npy')
  y_train_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/'+attribute+'_mask.npy')

  x_train_loaded_filtered, y_train_loaded_filtered = filter(x_train_loaded,y_train_loaded)
  x_val_loaded_filtered, y_val_loaded_filtered = filter(x_val_loaded,y_val_loaded)

  y_val_pred = original_models[attribute].predict(x_val_loaded_filtered)
  y_val_pred_new = optimal_models[attribute].predict(x_val_loaded_filtered)
  
  threshold = calculateThreshold(y_train_loaded_filtered)

  y_val_pred_thresholded = getThresholded(y_val_pred, threshold)
  y_val_pred_new_thresholded = getThresholded(y_val_pred_new, threshold)

  y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)

  #mean_IOU = getMeanIOU(y_val_thresholded, y_val_pred_thresholded)
  #new_mean_IOU = getMeanIOU(y_val_thresholded, y_val_pred_new_thresholded)

  print("Threshold: "+str(threshold))

  print("IOUS: \n")
  # IOUS
  ious = getIOUs(y_val_thresholded, y_val_pred_thresholded)
  new_ious = getIOUs(y_val_thresholded, y_val_pred_new_thresholded)
  # Creating plot
  titles = ["Original","Optimized"]
  iou_list = [ious,new_ious]
  plt.figure(figsize=(20,5))
  for i in range(len(titles)):
    plt.subplot(1, len(titles), i + 1)
    plt.title(titles[i])
    plt.boxplot(iou_list[i])
  plt.show()
  print("Mean IOU Original: "+str(np.mean(ious)))
  print("Mean IOU Optimized: "+str(np.mean(new_ious)))
  print("Dice Coeffs: \n")
  # Dice Scores
  dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_thresholded)
  new_dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred_new_thresholded)

  # Creating plot
  titles = ["Original","Optimized"]
  dice_coeff_list = [dice_coeffs,new_dice_coeffs]
  plt.figure(figsize=(20,5))
  for i in range(len(titles)):
    plt.subplot(1, len(titles), i + 1)
    plt.title(titles[i])
    plt.boxplot(dice_coeff_list[i])
  plt.show()
  print("Mean Dice Original: "+str(np.mean(dice_coeffs)))
  print("Mean Dice Optimized: "+str(np.mean(new_dice_coeffs)))

  for i in range(len(y_val_pred)):
    print("Original:")
    visualize(image=x_val_loaded_filtered[i].squeeze(),mask=y_val_loaded_filtered[i].squeeze(),prediction=y_val_pred[i].squeeze(),thresholded_pred=y_val_pred_thresholded[i].squeeze())
    print("Optimized:")
    visualize(image=x_val_loaded_filtered[i].squeeze(),mask=y_val_loaded_filtered[i].squeeze(),prediction=y_val_pred_new[i].squeeze(),thresholded_pred=y_val_pred_new_thresholded[i].squeeze())
    #overlay(image=x_val_loaded_filtered[i].squeeze(),mask=y_val_loaded_filtered[i].squeeze())
    print(sorted([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"])[i])
    print("\n")

# **RESULTS**

# **GLOBULES**

In [ ]:
visualize_attribute('globules')

# **MILIA LIKE CYST**

In [ ]:
visualize_attribute('milia_like_cyst')

# **PIGMENT NETWORK**

In [ ]:
# lr_-4 and full_filters
visualize_attribute('pigment_network')

In [ ]:
load_and_display("lr_-4/full_filters")

In [ ]:
loa

In [ ]:
load_and_display("additional_block")

In [ ]:
# dropout slightly improved meaniou but not meandice, visually looks better
load_and_display("additional_block_dropout")

In [ ]:
# this did worse so increasing filters prob won't help
load_and_display("lr_-4/1.2_filters")

# **NEGATIVE NETWORK**

In [ ]:
visualize_attribute('negative_network')

# **STREAKS**

In [ ]:
visualize_attribute('streaks')

# A mess: old collection

In [ ]:
threshold = calculateThreshold(y_val_loaded_filtered)
y_val_pred_thresholded = getThresholded(y_val_pred, threshold)
y_val_thresholded = getThresholded(y_val_loaded_filtered, threshold)
print(threshold)

In [ ]:
# Play around with thresholding

# only train on the ones with positive masks

# proportional of pixels that are positive out of all pixels in masks

y_val_pred = y_val_pred_streaks > 0.01
y_val_pred = y_val_pred * 1
y_val_thresholded = y_val_loaded_streaks >= 1
y_val_thresholded = y_val_thresholded

In [ ]:
print(np.max(y_val_pred_streaks))

In [ ]:
for i in range(len(x_val_loaded_filtered)):
  visualize(image=x_val_loaded_filtered[i].squeeze(),mask=y_val_thresholded[i].squeeze(),pred=y_val_pred_thresholded[i].squeeze())
  print([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"][i])

In [ ]:
x_test_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/test_data.npy')
y_test_pred=model.predict(x_test)


In [ ]:
meanIOU = getMeanIOU(y_val_thresholded, y_val_pred)

print("Mean IoU =", meanIOU)

dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred)

# Creating plot
plt.boxplot(dice_coeffs)
 
# show plot
plt.show()

In [ ]:
#Load previously saved milia like cysts model
from keras.models import load_model
model = load_model("drive/MyDrive/PlusDS/savedModels/milia_like_cystModel.h5", compile=False)
# Milia list cysts - predictions without thresholding
x_val_loaded = np.load('/content/drive/My Drive/PlusDS/processedData/val_data.npy')
y_val_loaded_milia = np.load('/content/drive/My Drive/PlusDS/processedData/milia_like_cyst_val_mask.npy')

y_val_pred_milia = model.predict(x_val_loaded)
for i in range(len(y_val_pred_milia)):
  visualize(image=x_val_loaded[i].squeeze(),mask=y_val_loaded_milia[i].squeeze(),prediction=y_val_pred_milia[i].squeeze())
  print(sorted([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"])[i])

In [ ]:
# Play around with thresholding
y_val_pred = y_val_pred_milia > 0.027
y_val_pred = y_val_pred * 1
y_val_thresholded = y_val_loaded_milia >= 1
y_val_thresholded = y_val_thresholded
for i in range(len(x_val_loaded)):
  visualize(image=x_val_loaded[i].squeeze(),mask=y_val_loaded_milia[i].squeeze(),pred=y_val_pred[i].squeeze())
  print([id for id in os.listdir("/content/drive/My Drive/PlusDS/ISIC2018_Task1-2_Validation_Input") if id[-4:] == ".jpg"][i])

In [ ]:
print(np.max(y_val_pred_milia))

In [ ]:

meanIOU = getMeanIOU(y_val_thresholded, y_val_pred)

print("Mean IoU =", meanIOU)

dice_coeffs = getDiceCoeffs(y_val_thresholded, y_val_pred)

# Creating plot
plt.boxplot(dice_coeffs)
 
# show plot
plt.show()

OLD- uneccessary but may use to improve in future

In [ ]:
# visualizing data

#dataset = Dataset(x_train_dir, y_train_dir)
#image, mask = dataset[0]
#image, masks = dataset[0] # get some sample
#print(masks["globules_mask"])
visualize(
    image=x_test[10], 
    #globules_mask=mask.squeeze()
    #globules_mask=masks["globules_mask"].squeeze(),
    #pigment_network_mask=masks["pigment_network_mask"].squeeze(),
    #milia_like_cyst_mask=masks["milia_like_cyst_mask"].squeeze(),
    #negative_network_mask=masks["negative_network_mask"].squeeze(),
    #streaks_mask=masks["streaks_mask"].squeeze()
)

In [ ]:
# Unused augmentation functions, may look at in future to improve
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        #albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.augmentations.transforms.GaussNoise(p=0.2),
        albu.augmentations.geometric.transforms.Perspective(p=0.5),
        

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.augmentations.transforms.Sharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        #albu.augmentations.transforms.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:

#### Visualize resulted augmented images and masks

augmented_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
)

# same image with different random transforms
for i in range(3):
    image, mask = augmented_dataset[1]
    #image, masks = augmented_dataset[1]
    #print(masks["globules_mask"])
    visualize(image=image, 
              globules_mask=mask.squeeze()
              #globules_mask=masks["globules_mask"].squeeze(),
              #pigment_network_mask=masks["pigment_network_mask"].squeeze(),
              #milia_like_cyst_mask=masks["milia_like_cyst_mask"].squeeze(),
              #negative_network_mask=masks["negative_network_mask"].squeeze(),
              #streaks_mask=masks["streaks_mask"].squeeze()
              )

In [ ]:
# OLD CAN DELETE AFTER TESTING NEW LOOPER AND FUNCTIONS
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

# Training Data
dataset = Dataset(x_train_dir, y_train_dir,classes=['globules'])
x_train = []
y_train = []

for i in range(len(dataset)):
  image, mask = dataset[i]
  x_train.append(image)
  y_train.append(mask)

# Resolving numpy array of list of numpy arrays not detecting shape issue
x_train = np.concatenate( [x_train], axis=0 )
y_train = np.concatenate( [y_train], axis=0 )


# Adding dimension to mask
y_train = np.expand_dims(y_train, axis=-1)

# Validation Data
val_dataset = Dataset(x_valid_dir, y_valid_dir,classes=['globules'])
x_val = []
y_val = []
for i in range(len(val_dataset)):
  image, mask = val_dataset[i]
  x_val.append(image)
  y_val.append(mask)

x_val = np.concatenate( [x_val], axis=0 )
y_val = np.concatenate( [y_val], axis=0 )

# Adding dimension to mask
y_val = np.expand_dims(y_val, axis=-1)



In [ ]:
# OLD Normalization
print(x_train[0][0][0])
x_train = x_train /255
y_train = y_train/255
x_val = x_val/255
y_val = y_val/255
x_test = x_test/255
print(x_train[0][0][0])
print("train data shape is: ", y_val.shape)
print("val data shape is: ", x_val.shape)

In [ ]:
IMG_HEIGHT = x_train.shape[1]
IMG_WIDTH  = x_train.shape[2]
IMG_CHANNELS = x_train.shape[3]

input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [ ]:
# define model
#model = Unet(BACKBONE, encoder_weights='imagenet')


model = build_unet(input_shape, n_classes=1)
model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])

# fit model
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=1,
    epochs=100,
    validation_data=(x_val, y_val),
)

In [ ]:
model.save("firstModel.h5")

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
x_test = []
test_dataset = Dataset(x_test_dir)
for i in range(len(dataset)):
  image, mask = dataset[i]
  x_test.append(image)

x_test = np.concatenate( [x_test], axis=0 )

In [ ]:
#IOU
y_pred=model.predict(x_test)
y_pred_thresholded = y_pred > 0.5

In [ ]:
y_val_pred = model.predict(x_val)
y_val_pred_thresholded = y_val_pred > 0.5
y_val_pred_thresholded = y_val_pred_thresholded*1
y_val_thresholded = y_val > 0.5
y_val_thresholded = y_val_thresholded*1

In [ ]:
from tensorflow.keras.metrics import MeanIoU

In [ ]:
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_val_thresholded, y_val_pred_thresholded)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
# Dice Coefficient
dice_coefs = []
print(len(y_val_thresholded))
for i in range(len(y_val_pred_thresholded)):
  im_sum = y_val_pred_thresholded[i].sum() + y_val_thresholded[i].sum()
  if im_sum != 0:
    # Compute Dice coefficient
    intersection = np.logical_and(y_val_pred_thresholded[i], y_val_thresholded[i])
    dice_coefs.append(2.0 * intersection.sum() / im_sum)
  else:
    dice_coefs.append(0)
print(dice_coefs)

In [ ]:
# Creating plot
plt.boxplot(dice_coefs)
 
# show plot
plt.show()

In [ ]:
import random
threshold = 0.5
test_img_number = random.randint(0, len(x_val)-1)
test_img = x_val[test_img_number]
ground_truth=y_val[test_img_number]
test_img_input=np.expand_dims(test_img, 0)
print(test_img_input.shape)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
print(prediction.shape)

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0])
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction, cmap='gray')

plt.show()

In [ ]:
#Load previously saved model
from keras.models import load_model
model = load_model("/drive/MyDrive/PlusDS/firstModel.hdf5", compile=False)

In [ ]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

In [ ]:
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['globules']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    preprocessing=get_preprocessing(preprocessing_fn)
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn)
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')